In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
pwd

'c:\\Users\\shrin\\Documents\\Python_Scripts\\FetchSearch'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig():
    root_dir: Path
    data_path: Path

In [5]:
from fetchSearch.constants import *
from fetchSearch.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path
        )

        return data_transformation_config

In [8]:
import os
from fetchSearch.logging import logger
from sentence_transformers import InputExample, evaluation
from datasets import load_dataset, load_from_disk
import torch
from torch.utils.data import DataLoader

In [19]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config


    def get_dataloader(self, data, split):
        examples = []
        data = data[split]
        n_examples = data.num_rows

        for i in range(n_examples):
            example = data[i]
            examples.append(InputExample(texts=[example['offer_ext'], example['search_query']], label=float(example["score"])))
        logger.info(f"in {split}, We have a {type(examples)} of length {len(examples)} containing {type(examples[0])}'s.")
        dataloader = DataLoader(examples, shuffle=True, batch_size=16)
        return examples, dataloader

    def convert(self):
        dataset = load_from_disk(self.config.data_path)

        
        train_examples, train_dataloader = self.get_dataloader(dataset, "train")
        torch.save(train_dataloader, os.path.join(self.config.root_dir,"train.pth"))

        
        val_examples, val_dataloader = self.get_dataloader(dataset, "val")
        val_evaluator = evaluation.EmbeddingSimilarityEvaluator([],[],[]).from_input_examples(examples=val_examples)
        torch.save(val_dataloader, os.path.join(self.config.root_dir,"val.pth"))
        torch.save(val_evaluator, os.path.join(self.config.root_dir,"val_eval.pth"))

        test_examples, test_dataloader = self.get_dataloader(dataset, "test")
        test_evaluator = evaluation.EmbeddingSimilarityEvaluator([],[],[]).from_input_examples(examples=test_examples)
        torch.save(test_dataloader, os.path.join(self.config.root_dir,"test.pth"))
        torch.save(test_evaluator, os.path.join(self.config.root_dir,"test_eval.pth"))


In [20]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2023-10-09 11:22:25,682: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-09 11:22:25,684: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-09 11:22:25,685: INFO: common: created directory at: artifacts]
[2023-10-09 11:22:25,687: INFO: common: created directory at: artifacts/data_transformation]
[2023-10-09 11:22:26,211: INFO: 2082942707: in train, We have a <class 'list'> of length 7314 containing <class 'sentence_transformers.readers.InputExample.InputExample'>'s.]
[2023-10-09 11:22:26,465: INFO: 2082942707: in val, We have a <class 'list'> of length 2814 containing <class 'sentence_transformers.readers.InputExample.InputExample'>'s.]
[2023-10-09 11:22:26,578: INFO: 2082942707: in test, We have a <class 'list'> of length 1125 containing <class 'sentence_transformers.readers.InputExample.InputExample'>'s.]
